In [ ]:
# setx HF_HOME "E:\huggingface"
# pip install --upgrade huggingface_hub
# huggingface-cli login


In [1]:
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionXLPipeline,
    FluxPipeline
)
import torch
import torch_directml
from pathlib import Path


e:\pyWorkplace\fantasyGen\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Generator_2:

    def __init__(self):


        dml = torch_directml.device()

        # --- SETTINGS ---
        base_model = "runwayml/stable-diffusion-v1-5"  # SD1.5 model
        self.lora_path = "./SD"  # Local folder or HF repo
        lora_file = "FantasyClassics.safetensors"  # LoRA file name

        # --- LOAD BASE MODEL ---
        self.pipe = StableDiffusionPipeline.from_pretrained(
            base_model,
            torch_dtype=torch.float16
        )
        self.pipe = self.pipe.to(dml)

    def load_lora(self, lora_file):

        self.pipe.load_lora_weights(self.lora_path, weight_name=lora_file)
        self.pipe.fuse_lora(lora_scale=0.8)  # Adjust 0.0-1.0 for LoRA strength

    def generate(self, prompt):
        image = self.pipe(prompt, guidance_scale=7.5, num_inference_steps=25).images[0]

        return image

In [3]:

class Generator:
    def __init__(self, base_model="runwayml/stable-diffusion-v1-5", model_type="sd15", local_file=False):
        self.dml = torch_directml.device()
        self.base_model = base_model
        self.model_type = model_type
        self.local_file = local_file
        self.lora_path = "./backend/resources/"

        self.pipe = self.load_pipeline()

    def load_pipeline(self):
        if self.local_file:
            model_path = Path(self.base_model)
            if not model_path.exists():
                raise FileNotFoundError(f"Model file not found: {model_path}")

            if self.model_type == "sd15":
                print("from file")
                pipe = StableDiffusionPipeline.from_single_file(model_path, torch_dtype=torch.float16)
            elif self.model_type == "sdxl":
                pipe = StableDiffusionXLPipeline.from_single_file(model_path, torch_dtype=torch.float16)
            else:
                pipe = FluxPipeline.from_single_file(model_path, torch_dtype=torch.float16)

        else:
            if self.model_type == "sd15":
                print("from pretrained")
                pipe = StableDiffusionPipeline.from_pretrained(self.base_model, torch_dtype=torch.float16)
            elif self.model_type == "sdxl":
                pipe = StableDiffusionXLPipeline.from_pretrained(self.base_model, torch_dtype=torch.float16)
            else:
                pipe = FluxPipeline.from_pretrained(self.base_model, torch_dtype=torch.float16)

        return pipe.to(self.dml)

    def load_lora(self, lora_file, scale=0.8):
        self.pipe.load_lora_weights(self.lora_path, weight_name=lora_file)
        self.pipe.fuse_lora(lora_scale=scale)

    def generate(self, prompt, guidance_scale=7.5, steps=25, width=None, height=None):
        kwargs = dict(
            prompt=prompt,
            guidance_scale=guidance_scale,
            num_inference_steps=steps
        )

        if width and height:
            kwargs["width"] = width
            kwargs["height"] = height

        image = self.pipe(**kwargs).images[0]
        return image

In [4]:
!pip install hf_xet


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
prompt = "big castle"

In [ ]:
# Load from local checkpoint
gen = Generator(base_model="./models/illustriousXL.safetensors", model_type="sdxl", local_file=True)

In [8]:
gen = Generator(base_model="black-forest-labs/FLUX.1-dev", model_type="auto")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

: 

In [9]:
gen = Generator(base_model="stabilityai/stable-diffusion-xl-base-1.0", model_type="sdxl")

Loading pipeline components...:  14%|█▍        | 1/7 [00:11<01:09, 11.61s/it]

: 

In [ ]:
gen = Generator(base_model="runwayml/stable-diffusion-v1-5", model_type="sd15")


from pretrained


Fetching 15 files:   7%|▋         | 1/15 [00:00<00:04,  3.33it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Loading pipeline components...: 100%|██████████| 7/7 [02:11<00:00, 18.82s/it]


: 

In [12]:
image = gen.generate(prompt=prompt)

image.save("output.png")


100%|██████████| 25/25 [01:28<00:00,  3.53s/it]


In [ ]:
gen_2 = Generator_2()

image = gen_2.generate(prompt=prompt)
image.save("./notebooks/file.png")


100%|██████████| 25/25 [02:26<00:00,  5.87s/it]


ValueError: unknown file extension: 